In [34]:
import pandas as pd
import numpy as np
import math
import geopandas as gpd
from shapely.geometry import Point

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("Clean_Process_data"), '..', 'functions/data_transforming')))
from spatial_analysis import *

# Clip zip code shapefile to only SF

In [35]:
communities = gpd.read_file('../data/raw_data/zip_codes/geo_export_37348335-ac06-4ade-bf6d-453d393d539d.shp')

In [36]:
communities = communities[communities["po_name"] == "SAN FRANCISCO"]
communities = communities[communities["zip"] != '94130']
communities = communities[communities["zip"] != '94129']
communities = communities[communities["zip"] != '94128']

In [37]:
communities.to_file('../data/clean_data/SF_zip_codes/SF_zip_codes.shp')

# Clean/Process parks data

In [38]:
parks = pd.read_csv('../data/raw_data/parks_data.csv')

In [39]:
parks = parks.iloc[1:,:]
parks = parks.dropna(subset = ["Location 1"])

In [40]:
temp = parks["Location 1"].apply(lambda x: x.split('\n'))
parks["address"] = temp.apply(lambda x: x[0])
parks["city_state"] = temp.apply(lambda x: x[1])
parks["lat_long"] = temp.apply(lambda x: x[2])
del parks["Location 1"]
del parks["Lat"]
del parks["Zipcode"]

In [41]:
# Add zip code field to PARKS data using spatial join
parks = spatial_join_zipcode(parks, communities)

In [42]:
parks.to_csv('../data/clean_data/parks.csv', index=False)

# Clean/Process schools data

In [43]:
schools = pd.read_csv('../data/raw_data/Schools.csv')

In [44]:
schools = schools.dropna(subset = ["Location 1"])
temp = schools["Location 1"].apply(lambda x: x.split('\n'))
schools["State"] = temp.apply(lambda x: x[0])
schools["lat_long"] = temp.apply(lambda x: x[1])
del schools["Location 1"]
del schools["State"]

In [45]:
# Add zip code field to SCHOOLS data using spatial join
schools = spatial_join_zipcode(schools, communities)

In [46]:
schools.to_csv('../data/clean_data/schools.csv', index=False)

# Clean/Process bart stations data

In [47]:
bart_stations = gpd.read_file('../data/raw_data/bart_stations/BART_Sta_13.shp')

In [48]:
# Add zip code field to BART STATIONS data using spatial join
bart_stations = bart_stations.to_crs({'init': 'epsg:4326'})
bart_stations = gpd.sjoin(bart_stations, communities, how="inner", op='intersects')
del bart_stations["index_right"]

In [49]:
bart_stations.to_csv('../data/clean_data/bart_stations.csv', index=False)

# Process police department locations data

In [109]:
SFPD_locations = pd.read_csv('../data/raw_data/Police_Stations__2011_.csv')

In [113]:
SFPD_locations = SFPD_locations.rename(columns={'Location':'lat_long'})

In [114]:
# Add zip code field to SFPD LOCATIONS data using spatial join
SFPD_locations = spatial_join_zipcode(SFPD_locations, communities)

In [115]:
SFPD_locations.to_csv('../data/clean_data/Police_Stations.csv', index=False)

# Process police incident data

In [62]:
police_incidents = pd.read_csv('../data/raw_data/SFPD_Incidents_4052016.csv')

In [65]:
police_incidents = police_incidents.rename(columns={'Location':'lat_long'})

In [66]:
# add zip code field to POLICE INCIDENT data using spatial join
police_incidents = spatial_join_zipcode(police_incidents, communities)

In [69]:
police_incidents.to_csv('../data/clean_data/SFPD_Incidents_4052016.csv', index=False)

# Process zillow/citydata

In [70]:
zillow_citydata = pd.read_csv('../data/raw_data/zillow_citydata.csv')

In [100]:
zillow_citydata = zillow_citydata.rename(columns={'lontitude':'longitude'})
zillow_citydata["lat_long"] = zillow_citydata[["latitude", "longitude"]].apply(lambda x: str(tuple(x)), axis=1)

In [101]:
# add zip code field to ZILLOW/CITYDATA using spatial join
zillow_citydata = spatial_join_zipcode(zillow_citydata, communities)
del zillow_citydata["ZIP"]
del zillow_citydata["State"]
del zillow_citydata["City"]

In [107]:
zillow_citydata.to_csv('../data/clean_data/zillow_citydata.csv', index=False)